In [1]:
from openbabel import pybel
import os
import re
import argparse
import numpy as np
import pandas as pd
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem.rdMolAlign import AlignMol
import copy
from openbabel import openbabel as ob

In [2]:
directory = "/home/jpam/google_drive/UFMG/2020-1/modelagem/moleculas_desenhadas/QSAR/3D"
alignment_file = "alinhamento.csv"

In [3]:
df = pd.read_csv(os.path.join(directory,alignment_file),sep=";",header = None)

In [9]:
files = [f for f in os.listdir(directory) if f.endswith("mol2")]

In [4]:
mols = [pybel.readfile('mol2',os.path.join(directory,f)) 
	    for f in df.iloc[:,0]]

In [5]:
rdmols = []
for mol in mols:
    for c in mol:
        rdmol = Chem.MolFromMolBlock(c.write('mol'),removeHs=False)
    rdmols.append(copy.copy(rdmol))

In [ ]:
rdmols

In [6]:
refAtoms = [int(a)-1 for a in df[1][0].split(',')]

In [7]:
refMol = rdmols[0]
mols_aligned = [pybel.readstring("mol",Chem.MolToMolBlock(refMol))]
for i,prbMol in enumerate(rdmols[1:]):
    prbAtoms = [int(a)-1 for a in df[1][i+1].split(',')]
    alinhamento = AlignMol(prbMol,refMol,atomMap=list(zip(prbAtoms,refAtoms)))
    mols_aligned.append(pybel.readstring("mol",Chem.MolToMolBlock(prbMol)))


In [10]:
for i,(m1,m2) in enumerate(zip(mols_aligned,mols)):
    for (a1,a2) in zip(m1,m2):
        print(a1,a2)
        a1.partialcharge = a2.partialcharge
    m1.write("mol2",files[i][:-4]+"_aligned.mol2",True)

In [ ]:
obConversion = ob.OBConversion()

In [ ]:
obConversion.SetInAndOutFormats("g09","mol2")

In [ ]:
obmols = []
for i in range(len(files)):
    obmol = ob.OBMol()
    obConversion.ReadFile(obmol, os.path.join(directory,df.iloc[i,0]+".log"))
    obmols.append(obmol)

In [ ]:
os.path.basename(obmols[0].GetTitle())

In [ ]:
for at in ob.OBMolAtomIter(obmols[0]):
    print(at.GetPartialCharge())

In [ ]:
for m1,m2 in zip(mols_aligned,obmols):
    for (a1,a2) in zip(ob.OBMolAtomIter(m1.OBMol),ob.OBMolAtomIter(m2)):
#         a1.partialcharge = a2.partialcharge
        a1.SetPartialCharge(a2.GetPartialCharge())
    m1.write("mol2",m1.title[:-4]+"_aligned.mol2",True)

In [ ]:
obmols[0].GetTitle()

In [ ]:
m1.title